## 1. 환경 설정

`(1) Env 환경변수`

In [1]:
print('he')

he


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [3]:
import re
import os, json
from glob import glob

from textwrap import dedent
from pprint import pprint

import uuid

import warnings
warnings.filterwarnings("ignore")

# 함수설정 

In [11]:
def parse_law(law_text):
    # 서문 분리
    # '^'로 시작하여 '제1장' 또는 '제1조' 직전까지의 모든 텍스트를 탐색 
    preamble_pattern = r'^(.*?)(?=제1장|제1조)'
    preamble = re.search(preamble_pattern, law_text, re.DOTALL)
    if preamble:
        preamble = preamble.group(1).strip()
    
    # 장 분리 
    # '제X장' 형식의 제목과 그 뒤에 오는 모든 조항을 하나의 그룹화 
    chapter_pattern = r'(제\d+장\s+.+?)\n((?:제\d+조(?:의\d+)?(?:\(\w+\))?.*?)(?=제\d+장|부칙|$))'
    chapters = re.findall(chapter_pattern, law_text, re.DOTALL)
    
    # 부칙 분리
    # '부칙'으로 시작하는 모든 텍스트를 탐색 
    appendix_pattern = r'(부칙.*)'
    appendix = re.search(appendix_pattern, law_text, re.DOTALL)
    if appendix:
        appendix = appendix.group(1)
    
    # 파싱 결과를 저장할 딕셔너리 초기화
    parsed_law = {'서문': preamble, '장': {}, '부칙': appendix}
    
    # 각 장 내에서 조 분리
    for chapter_title, chapter_content in chapters:
        # 조 분리 패턴
        # 1. '제X조'로 시작 ('제X조의Y' 형식도 가능)
        # 2. 조 번호 뒤에 반드시 '(항목명)' 형식의 제목이 와야 함 
        # 3. 다음 조가 시작되기 전까지 또는 문서의 끝까지의 모든 내용을 포함
        article_pattern = r'(제\d+조(?:의\d+)?\s*\([^)]+\).*?)(?=제\d+조(?:의\d+)?\s*\([^)]+\)|$)'
        
        # 정규표현식을 이용해 모든 조항을 탐색 
        articles = re.findall(article_pattern, chapter_content, re.DOTALL)
        
        # 각 조항의 앞뒤 공백을 제거하고 결과 딕셔너리에 저장
        parsed_law['장'][chapter_title.strip()] = [article.strip() for article in articles]
    
    return parsed_law

In [12]:
# 파싱 함수를 수정 (장이 없이 조문으로만 구성된 경우)
def parse_law_v2(law_text):
    # 서문 분리
    preamble_pattern = r'^(.*?)(?=제1장|제1조)'
    preamble = re.search(preamble_pattern, law_text, re.DOTALL)
    if preamble:
        preamble = preamble.group(1).strip()
    
    # 장 분리 
    chapter_pattern = r'(제\d+장\s+.+?)\n((?:제\d+조(?:의\d+)?(?:\(\w+\))?.*?)(?=제\d+장|부칙|$))'
    chapters = re.findall(chapter_pattern, law_text, re.DOTALL)
    
    # 부칙 분리
    appendix_pattern = r'(부칙.*)'
    appendix = re.search(appendix_pattern, law_text, re.DOTALL)
    if appendix:
        appendix = appendix.group(1)
    
    parsed_law = {'서문': preamble, '부칙': appendix}
    
    # 조 분리 패턴
    article_pattern = r'(제\d+조(?:의\d+)?\s*\([^)]+\).*?)(?=제\d+조(?:의\d+)?\s*\([^)]+\)|$)'
    
    if chapters:  # 장이 있는 경우
        parsed_law['장'] = {}
        for chapter_title, chapter_content in chapters:
            articles = re.findall(article_pattern, chapter_content, re.DOTALL)
            parsed_law['장'][chapter_title.strip()] = [article.strip() for article in articles]
    else:  # 장이 없는 경우
        # 서문과 부칙을 제외한 본문에서 조문 추출
        main_text = re.sub(preamble_pattern, '', law_text, flags=re.DOTALL)
        main_text = re.sub(appendix_pattern, '', main_text, flags=re.DOTALL)
        articles = re.findall(article_pattern, main_text, re.DOTALL)
        parsed_law['조문'] = [article.strip() for article in articles]
    
    return parsed_law

## 2. 법률 문서를 로드하여 벡터저장소에 저장

In [4]:
os.getcwd()

'/Users/simsewon/Documents/agent_project/NaturalLangProcessingInBigData/notebook'

In [5]:
os.chdir('/Users/simsewon/Documents/agent_project/NaturalLangProcessingInBigData')

In [6]:
# pdf 파일 목록을 확인
pdf_files = glob(os.path.join('data', '*.pdf'))

pdf_files

['data/근로기준법(법률)(제18176호)(20211119).pdf',
 'data/전자상거래 등에서의 소비자보호에 관한 법률(법률)(제20302호)(20250214).pdf',
 'data/도로교통법(법률)(제20375호)(20250320).pdf',
 'data/개인정보 보호법(법률)(제19234호)(20240315).pdf',
 'data/주택임대차보호법(법률)(제19356호)(20230719).pdf']

###  개인정보보호법

- PDf 문서를 가져와서 조항 별로 구분하여 정리

In [55]:
from langchain_community.document_loaders import PyPDFLoader

pdf_file = 'data/개인정보 보호법(법률)(제19234호)(20240315).pdf'

loader = PyPDFLoader(pdf_file)
pages = loader.load()

len(pages)

41

In [56]:
pages[0].page_content

'법제처                                                            1                                                       국가법령정보센터\n개인정보 보호법 \n \n개인정보 보호법 \n[시행 2024. 3. 15.] [법률 제19234호, 2023. 3. 14., 일부개정] \n개인정보보호위원회 (심사총괄담당관 - 일반 법령해석) 02-2100-3043 \n개인정보보호위원회 (국제협력담당관 - 국외이전) 02-2100-2484, 2499 \n개인정보보호위원회 (개인정보보호정책과 - 법령 제ㆍ개정, 아동ㆍ청소년) 02-2100-3057, 3047 \n개인정보보호위원회 (신기술개인정보과 - 영상정보, 안전조치) 02-2100-3064, 3028 \n개인정보보호위원회 (데이터안전정책과 - 가명정보, 개인정보안심구역) 02-2100-3088, 3074, 3058, 3079 \n개인정보보호위원회 (자율보호정책과 - 보호책임자, 자율규제, 보호수준 평가, 처리방침, 영향평가) 02-2100-3083, 3089, 3087, 3096,\n3086 \n개인정보보호위원회 (분쟁조정과 - 분쟁조정, 손해배상책임) 1833-6972, 02-2100-3142 \n개인정보보호위원회 (범정부마이데이터 추진단(전략기획팀 - 전송요구권(마이데이터)) 02-2100-3173 \n       제1장 총칙\n \n제1조(목적) 이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자유와 권리를 보호하고, 나아가 개인\n의 존엄과 가치를 구현함을 목적으로 한다. <개정 2014. 3. 24.>\n \n제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2014. 3. 24., 2020. 2. 4., 2023. 3. 14.>\n1. “개인정보”란 살아 있는 개인에 관한 정보로서 다음 각 목의 어느 하나에 해당하는 정보를 말한다.\n가. 성명, 주민등록번호 및 영상 등을 통하여 개

In [57]:
pages[0].metadata

{'source': 'data/개인정보 보호법(법률)(제19234호)(20240315).pdf',
 'page': 0}

In [58]:
print(pages[0].page_content)

법제처                                                            1                                                       국가법령정보센터
개인정보 보호법 
 
개인정보 보호법 
[시행 2024. 3. 15.] [법률 제19234호, 2023. 3. 14., 일부개정] 
개인정보보호위원회 (심사총괄담당관 - 일반 법령해석) 02-2100-3043 
개인정보보호위원회 (국제협력담당관 - 국외이전) 02-2100-2484, 2499 
개인정보보호위원회 (개인정보보호정책과 - 법령 제ㆍ개정, 아동ㆍ청소년) 02-2100-3057, 3047 
개인정보보호위원회 (신기술개인정보과 - 영상정보, 안전조치) 02-2100-3064, 3028 
개인정보보호위원회 (데이터안전정책과 - 가명정보, 개인정보안심구역) 02-2100-3088, 3074, 3058, 3079 
개인정보보호위원회 (자율보호정책과 - 보호책임자, 자율규제, 보호수준 평가, 처리방침, 영향평가) 02-2100-3083, 3089, 3087, 3096,
3086 
개인정보보호위원회 (분쟁조정과 - 분쟁조정, 손해배상책임) 1833-6972, 02-2100-3142 
개인정보보호위원회 (범정부마이데이터 추진단(전략기획팀 - 전송요구권(마이데이터)) 02-2100-3173 
       제1장 총칙
 
제1조(목적) 이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자유와 권리를 보호하고, 나아가 개인
의 존엄과 가치를 구현함을 목적으로 한다. <개정 2014. 3. 24.>
 
제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2014. 3. 24., 2020. 2. 4., 2023. 3. 14.>
1. “개인정보”란 살아 있는 개인에 관한 정보로서 다음 각 목의 어느 하나에 해당하는 정보를 말한다.
가. 성명, 주민등록번호 및 영상 등을 통하여 개인을 알아볼 수 있는 정보 
나. 해당 

In [59]:



# 각 페이지의 텍스트를 결합하여 재분리
text_for_delete = r"법제처\s+\d+\s+국가법령정보센터\n개인정보 보호법"

law_text = "\n".join([re.sub(text_for_delete, "", p.page_content).strip() for p in pages])

parsed_law = parse_law(law_text)

# 분할된 아이템 갯수 확인
print(len(parsed_law["장"]))

10


In [154]:
parsed_law.keys()

dict_keys(['서문', '부칙', '조문'])

- 랭체인 Document 객체에 메타데이터와 함께 정리

In [61]:
from langchain_core.documents import Document

final_docs = []
for law in parsed_law['장'].keys():
    for article in parsed_law['장'][law]:

        # metadata 내용을 정리 
        metadata = {
                "source": pdf_file,
                "chapter": law,
                "name" : "개인정보 보호법"
                }

        # metadata 내용을 본문에 추가 
        content = f"[법률정보]\n다음 조항은 {metadata['name']} {metadata['chapter']}에서 발췌한 내용입니다.\n\n[법률조항]\n{article}"

        final_docs.append(Document(page_content=content, metadata=metadata))
        
len(final_docs)


109

In [62]:
print(final_docs[0].page_content)
print()


[법률정보]
다음 조항은 개인정보 보호법 제1장 총칙에서 발췌한 내용입니다.

[법률조항]
제1조(목적) 이 법은 개인정보의 처리 및 보호에 관한 사항을 정함으로써 개인의 자유와 권리를 보호하고, 나아가 개인
의 존엄과 가치를 구현함을 목적으로 한다. <개정 2014. 3. 24.>



In [63]:
print(final_docs[0].metadata)

{'source': 'data/개인정보 보호법(법률)(제19234호)(20240315).pdf', 'chapter': '제1장 총칙', 'name': '개인정보 보호법'}


In [64]:
print(final_docs[-1].page_content)
print()
print(final_docs[-1].metadata)

[법률정보]
다음 조항은 개인정보 보호법 제10장 벌칙 <개정 2020. 2. 4.>에서 발췌한 내용입니다.

[법률조항]
제76조(과태료에 관한 규정 적용의 특례) 제75조의 과태료에 관한 규정을 적용할 때 제64조의2에 따라 과징금을 부과
한 행위에 대하여는 과태료를 부과할 수 없다. <개정 2023. 3. 14.>
[본조신설 2013. 8. 6.]

{'source': 'data/개인정보 보호법(법률)(제19234호)(20240315).pdf', 'chapter': '제10장 벌칙 <개정 2020. 2. 4.>', 'name': '개인정보 보호법'}


- 벡터저장소에 인덱싱

In [65]:
# 각 문서의 텍스트 길이를 확인

text_lengths = [len(d.page_content) for d in final_docs]
print(min(text_lengths), max(text_lengths))

116 2844


In [70]:
from langchain_chroma import Chroma
from langchain_ollama  import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3") 

In [77]:
! pwd

3155.65s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/Users/simsewon/Library/Mobile Documents/com~apple~CloudDocs/agent_project/NaturalLangProcessingInBigData


In [78]:

# Chroma 인덱스 생성
personal_db = Chroma.from_documents(
    documents=final_docs, 
    embedding=embeddings_model,   
    collection_name="personal_law",
    persist_directory="../chroma_db",  # NaturalLangProcessingInBigData 경로 저장 
)

###  근로기준법

- PDf 문서를 가져와서 조항 별로 구분하여 정리

In [81]:
from langchain_community.document_loaders import PyPDFLoader

pdf_file = 'data/근로기준법(법률)(제18176호)(20211119).pdf'

loader = PyPDFLoader(pdf_file)
pages = loader.load()

len(pages)

20

In [82]:
pages[0].page_content

'법제처                                                            1                                                       국가법령정보센터\n근로기준법 \n \n근로기준법 \n[시행 2021. 11. 19.] [법률 제18176호, 2021. 5. 18., 일부개정] \n고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534 \n고용노동부 (근로기준정책과 - 소년) 044-202-7535 \n고용노동부 (근로기준정책과 - 임금) 044-202-7548 \n고용노동부 (여성고용정책과 - 여성) 044-202-7475 \n고용노동부 (임금근로시간정책과 - 근로시간, 휴게) 044-202-7545 \n고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973 \n고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530 \n고용노동부 (임금근로시간정책과 - 유연근로시간제) 044-202-7549 \n       제1장 총칙\n \n제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는\n국민경제의 발전을 꾀하는 것을 목적으로 한다.\n \n제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2018. 3. 20., 2019. 1. 15., 2020. 5. 26.>\n1. “근로자”란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 사람을 말한다.\n2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를\n말한다. \n3. “근로”란 정신노동과 육체노동을 말한다.\n4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체\n결된 계약을 말한다. \n5. “임금”이란 사용자가 근로의 대가로 근

In [83]:
# 각 페이지의 텍스트를 결합하여 재분리
text_for_delete = r"법제처\s+\d+\s+국가법령정보센터\n근로기준법"

law_text = "\n".join([re.sub(text_for_delete, "", p.page_content).strip() for p in pages])

parsed_law = parse_law(law_text)

# 분할된 아이템 갯수 확인
print(len(parsed_law["장"]))

12


In [84]:
parsed_law

{'서문': '근로기준법 \n[시행 2021. 11. 19.] [법률 제18176호, 2021. 5. 18., 일부개정] \n고용노동부 (근로기준정책과 - 해고, 취업규칙, 기타) 044-202-7534 \n고용노동부 (근로기준정책과 - 소년) 044-202-7535 \n고용노동부 (근로기준정책과 - 임금) 044-202-7548 \n고용노동부 (여성고용정책과 - 여성) 044-202-7475 \n고용노동부 (임금근로시간정책과 - 근로시간, 휴게) 044-202-7545 \n고용노동부 (임금근로시간정책과 - 휴일, 연차휴가) 044-202-7973 \n고용노동부 (임금근로시간정책과 - 제63조 적용제외, 특례업종) 044-202-7530 \n고용노동부 (임금근로시간정책과 - 유연근로시간제) 044-202-7549',
 '장': {'제1장 총칙': ['제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는\n국민경제의 발전을 꾀하는 것을 목적으로 한다.',
   '제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2018. 3. 20., 2019. 1. 15., 2020. 5. 26.>\n1. “근로자”란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 사람을 말한다.\n2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를\n말한다. \n3. “근로”란 정신노동과 육체노동을 말한다.\n4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체\n결된 계약을 말한다. \n5. “임금”이란 사용자가 근로의 대가로 근로자에게 임금, 봉급, 그 밖에 어떠한 명칭으로든지 지급하는 모든 금품을\n말한다. \n6. “평균임금”이란 이를 산정하여야 할 사유가 발생한 날 이전 3개월 동안에 그 근로자에게 지급된 임금의 총액을\n그 기간의 총일수로 나눈 금액

- 랭체인 Document 객체에 메타데이터와 함께 정리

In [85]:
from langchain_core.documents import Document

final_docs = []
for law in parsed_law['장'].keys():
    for article in parsed_law['장'][law]:

        # metadata 내용을 정리 
        metadata = {
                "source": pdf_file,
                "chapter": law,
                "name" : "근로기준법"
                }

        # metadata 내용을 본문에 추가 
        content = f"[법률정보]\n다음 조항은 {metadata['name']} {metadata['chapter']}에서 발췌한 내용입니다.\n\n[법률조항]\n{article}"

        final_docs.append(Document(page_content=content, metadata=metadata))
        
len(final_docs)

88

In [86]:
print(final_docs[0].page_content)
print()
print(final_docs[0].metadata)

[법률정보]
다음 조항은 근로기준법 제1장 총칙에서 발췌한 내용입니다.

[법률조항]
제1조(목적) 이 법은 헌법에 따라 근로조건의 기준을 정함으로써 근로자의 기본적 생활을 보장, 향상시키며 균형 있는
국민경제의 발전을 꾀하는 것을 목적으로 한다.

{'source': 'data/근로기준법(법률)(제18176호)(20211119).pdf', 'chapter': '제1장 총칙', 'name': '근로기준법'}


In [87]:
print(final_docs[1].page_content)
print()
print(final_docs[1].metadata)

[법률정보]
다음 조항은 근로기준법 제1장 총칙에서 발췌한 내용입니다.

[법률조항]
제2조(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2018. 3. 20., 2019. 1. 15., 2020. 5. 26.>
1. “근로자”란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 사람을 말한다.
2. “사용자”란 사업주 또는 사업 경영 담당자, 그 밖에 근로자에 관한 사항에 대하여 사업주를 위하여 행위하는 자를
말한다. 
3. “근로”란 정신노동과 육체노동을 말한다.
4. “근로계약”이란 근로자가 사용자에게 근로를 제공하고 사용자는 이에 대하여 임금을 지급하는 것을 목적으로 체
결된 계약을 말한다. 
5. “임금”이란 사용자가 근로의 대가로 근로자에게 임금, 봉급, 그 밖에 어떠한 명칭으로든지 지급하는 모든 금품을
말한다. 
6. “평균임금”이란 이를 산정하여야 할 사유가 발생한 날 이전 3개월 동안에 그 근로자에게 지급된 임금의 총액을
그 기간의 총일수로 나눈 금액을 말한다. 근로자가 취업한 후 3개월 미만인 경우도 이에 준한다. 
7. “1주”란 휴일을 포함한 7일을 말한다.
8. “소정(所定)근로시간”이란 제50조, 제69조 본문 또는 「산업안전보건법」 제139조제1항에 따른 근로시간의 범위에
서 근로자와 사용자 사이에 정한 근로시간을 말한다. 
9. “단시간근로자”란 1주 동안의 소정근로시간이 그 사업장에서 같은 종류의 업무에 종사하는 통상 근로자의 1주
동안의 소정근로시간에 비하여 짧은 근로자를 말한다. 
② 제1항제6호에 따라 산출된 금액이 그 근로자의 통상임금보다 적으면 그 통상임금액을 평균임금으로 한다.

{'source': 'data/근로기준법(법률)(제18176호)(20211119).pdf', 'chapter': '제1장 총칙', 'name': '근로기준법'}


- 벡터저장소에 인덱싱

In [88]:
# 각 문서의 텍스트 길이를 확인

text_lengths = [len(d.page_content) for d in final_docs]
print(min(text_lengths), max(text_lengths))

97 1547


In [89]:
from langchain_chroma import Chroma
from langchain_ollama  import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3") 

# Chroma 인덱스 생성
labor_db = Chroma.from_documents(
    documents=final_docs, 
    embedding=embeddings_model,   
    collection_name="labor_law",
    persist_directory="../chroma_db",
)

###  주택임대차보호법

- PDf 문서를 가져와서 조항 별로 구분하여 정리

In [90]:
from langchain_community.document_loaders import PyPDFLoader

pdf_file = 'data/주택임대차보호법(법률)(제19356호)(20230719).pdf'

loader = PyPDFLoader(pdf_file)
pages = loader.load()

len(pages)



10

In [92]:
# 각 페이지의 텍스트를 결합하여 재분리
text_for_delete = r"법제처\s+\d+\s+국가법령정보센터\n주택임대차보호법"

law_text = "\n".join([re.sub(text_for_delete, "", p.page_content).strip() for p in pages])

parsed_law = parse_law_v2(law_text)

# 분할된 아이템 갯수 확인
print(len(parsed_law["조문"]))

41


In [93]:
parsed_law

{'서문': '법제처                                                            1                                                       국가법령정보센터\n주택임대차보호법 \n \n주택임대차보호법 ( 약칭: 주택임대차법 ) \n[시행 2023. 7. 19.] [법률 제19356호, 2023. 4. 18., 일부개정] \n법무부 (법무심의관실) 02-2110-3164 \n국토교통부 (주택임대차기획팀) 044-201-3348, 4178',
 '부칙': '부칙 <제19520호,2023. 7. 11.> \n이 법은 공포한 날부터 시행한다.',
 '조문': ['제1조(목적) 이 법은 주거용 건물의 임대차(賃貸借)에 관하여 「민법」에 대한 특례를 규정함으로써 국민 주거생활의 안\n정을 보장함을 목적으로 한다.\n[전문개정 2008. 3. 21.]',
  '제2조(적용 범위) 이 법은 주거용 건물(이하 “주택”이라 한다)의 전부 또는 일부의 임대차에 관하여 적용한다. 그 임차주\n택(賃借住宅)의 일부가 주거 외의 목적으로 사용되는 경우에도 또한 같다.\n[전문개정 2008. 3. 21.]',
  '제3조(대항력 등) ① 임대차는 그 등기(登記)가 없는 경우에도 임차인(賃借人)이 주택의 인도(引渡)와 주민등록을 마친\n때에는 그 다음 날부터 제삼자에 대하여 효력이 생긴다. 이 경우 전입신고를 한 때에 주민등록이 된 것으로 본다.\n② 주택도시기금을 재원으로 하여 저소득층 무주택자에게 주거생활 안정을 목적으로 전세임대주택을 지원하는 법\n인이 주택을 임차한 후 지방자치단체의 장 또는 그 법인이 선정한 입주자가 그 주택을 인도받고 주민등록을 마쳤을\n때에는 제1항을 준용한다. 이 경우 대항력이 인정되는 법인은 대통령령으로 정한다.<개정 2015. 1. 6.>\n③ 「중소기업기본법」 제2조에 따른 중소기업에 해당하는 법인이 소속 직원의 주거용으로 주택을 임차한 후 그 법\n인이 선정한 직원이

- 랭체인 Document 객체에 메타데이터와 함께 정리

In [94]:
from langchain_core.documents import Document

final_docs = []
for article in parsed_law['조문']:

    # metadata 내용을 정리 
    metadata = {
            "source": pdf_file,
            "name" : "주택임대차보호법"
            }

    # metadata 내용을 본문에 추가 
    content = f"[법률정보]\n다음 조항은 {metadata['name']}에서 발췌한 내용입니다.\n\n[법률조항]\n{article}"

    final_docs.append(Document(page_content=content, metadata=metadata))

        
len(final_docs)

41

In [95]:
print(final_docs[0].page_content)
print()
print(final_docs[0].metadata)

[법률정보]
다음 조항은 주택임대차보호법에서 발췌한 내용입니다.

[법률조항]
제1조(목적) 이 법은 주거용 건물의 임대차(賃貸借)에 관하여 「민법」에 대한 특례를 규정함으로써 국민 주거생활의 안
정을 보장함을 목적으로 한다.
[전문개정 2008. 3. 21.]

{'source': 'data/주택임대차보호법(법률)(제19356호)(20230719).pdf', 'name': '주택임대차보호법'}


In [96]:
print(final_docs[1].page_content)
print()
print(final_docs[1].metadata)

[법률정보]
다음 조항은 주택임대차보호법에서 발췌한 내용입니다.

[법률조항]
제2조(적용 범위) 이 법은 주거용 건물(이하 “주택”이라 한다)의 전부 또는 일부의 임대차에 관하여 적용한다. 그 임차주
택(賃借住宅)의 일부가 주거 외의 목적으로 사용되는 경우에도 또한 같다.
[전문개정 2008. 3. 21.]

{'source': 'data/주택임대차보호법(법률)(제19356호)(20230719).pdf', 'name': '주택임대차보호법'}


- 벡터저장소에 인덱싱

In [97]:
# 각 문서의 텍스트 길이를 확인

text_lengths = [len(d.page_content) for d in final_docs]
print(min(text_lengths), max(text_lengths))

123 1879


In [98]:
from langchain_chroma import Chroma
from langchain_ollama  import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3") 

# Chroma 인덱스 생성
housing_db = Chroma.from_documents(
    documents=final_docs, 
    embedding=embeddings_model,   
    collection_name="housing_law",
    persist_directory="../chroma_db",
)

### 전자상거래법

- PDf 문서를 가져와서 조항 별로 구분하여 정리

In [7]:
from langchain_community.document_loaders import PyPDFLoader

pdf_file = 'data/전자상거래 등에서의 소비자보호에 관한 법률(법률)(제20302호)(20250214).pdf'

loader = PyPDFLoader(pdf_file)
pages = loader.load()

len(pages)



18

In [9]:

pages[0].page_content


'법제처                                                            1                                                       국가법령정보센터\n전자상거래 등에서의 소비자보호에 관한 법률 \n \n전자상거래 등에서의 소비자보호에 관한 법률 ( 약칭: 전자상거래법 ) \n[시행 2025. 2. 14.] [법률 제20302호, 2024. 2. 13., 일부개정] \n공정거래위원회 (소비자거래정책과) 044-200-4454 \n       제1장 총칙 <개정 2012. 2. 17.>\n \n제1조(목적) 이 법은 전자상거래 및 통신판매 등에 의한 재화 또는 용역의 공정한 거래에 관한 사항을 규정함으로써 소\n비자의 권익을 보호하고 시장의 신뢰도를 높여 국민경제의 건전한 발전에 이바지함을 목적으로 한다.\n[전문개정 2012. 2. 17.]\n \n제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2012. 6. 1.>\n1. “전자상거래”란 전자거래(「전자문서 및 전자거래 기본법」 제2조제5호에 따른 전자거래를 말한다. 이하 같다)의\n방법으로 상행위(商行爲)를 하는 것을 말한다. \n2. “통신판매”란 우편ㆍ전기통신, 그 밖에 총리령으로 정하는 방법으로 재화 또는 용역(일정한 시설을 이용하거나\n용역을 제공받을 수 있는 권리를 포함한다. 이하 같다)의 판매에 관한 정보를 제공하고 소비자의 청약을 받아 재 \n화 또는 용역(이하 “재화등”이라 한다)을 판매하는 것을 말한다. 다만, 「방문판매 등에 관한 법률」 제2조제3호에 \n따른 전화권유판매는 통신판매의 범위에서 제외한다. \n3. “통신판매업자”란 통신판매를 업(業)으로 하는 자 또는 그와의 약정에 따라 통신판매업무를 수행하는 자를 말한\n다. \n4. “통신판매중개”란 사이버몰(컴퓨터 등과 정보통신설비를 이용하여 재화등을 거래할 수 있도록 설정된 가상의 영\n업장을 말한다. 이하 같다)의 이용을 허락하거나

In [13]:

# 각 페이지의 텍스트를 결합하여 재분리
text_for_delete = r"법제처\s+\d+\s+국가법령정보센터\전자상거래 등에서의 소비자보호에 관한 법률"

law_text = "\n".join([re.sub(text_for_delete, "", p.page_content).strip() for p in pages])

parsed_law = parse_law(law_text)
parsed_law

{'서문': '법제처                                                            1                                                       국가법령정보센터\n전자상거래 등에서의 소비자보호에 관한 법률 \n \n전자상거래 등에서의 소비자보호에 관한 법률 ( 약칭: 전자상거래법 ) \n[시행 2025. 2. 14.] [법률 제20302호, 2024. 2. 13., 일부개정] \n공정거래위원회 (소비자거래정책과) 044-200-4454',
 '장': {'제1장 총칙 <개정 2012. 2. 17.>': ['제1조(목적) 이 법은 전자상거래 및 통신판매 등에 의한 재화 또는 용역의 공정한 거래에 관한 사항을 규정함으로써 소\n비자의 권익을 보호하고 시장의 신뢰도를 높여 국민경제의 건전한 발전에 이바지함을 목적으로 한다.\n[전문개정 2012. 2. 17.]',
   '제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2012. 6. 1.>\n1. “전자상거래”란 전자거래(「전자문서 및 전자거래 기본법」 제2조제5호에 따른 전자거래를 말한다. 이하 같다)의\n방법으로 상행위(商行爲)를 하는 것을 말한다. \n2. “통신판매”란 우편ㆍ전기통신, 그 밖에 총리령으로 정하는 방법으로 재화 또는 용역(일정한 시설을 이용하거나\n용역을 제공받을 수 있는 권리를 포함한다. 이하 같다)의 판매에 관한 정보를 제공하고 소비자의 청약을 받아 재 \n화 또는 용역(이하 “재화등”이라 한다)을 판매하는 것을 말한다. 다만, 「방문판매 등에 관한 법률」 제2조제3호에 \n따른 전화권유판매는 통신판매의 범위에서 제외한다. \n3. “통신판매업자”란 통신판매를 업(業)으로 하는 자 또는 그와의 약정에 따라 통신판매업무를 수행하는 자를 말한\n다. \n4. “통신판매중개”란 사이버몰(컴퓨터 등과 정보통신설비를 이용하여 재화등을 거래할 수 있도록 설정된 가상의 영\n업장을 말한다. 이하 같다)의

In [14]:
# 분할된 아이템 갯수 확인
print(len(parsed_law["장"]))


7


In [16]:


# 랭체인 Document 객체에 메타데이터와 함께 정리
from langchain_core.documents import Document

final_docs = []
for law in parsed_law['장'].keys():
    for article in parsed_law['장'][law]:

        # metadata 내용을 정리 
        metadata = {
                "source": pdf_file,
                "chapter": law,
                "name" : "전자상거래소비자보호법"
                }

        # metadata 내용을 본문에 추가 
        content = f"[법률정보]\n다음 조항은 {metadata['name']} {metadata['chapter']}에서 발췌한 내용입니다.\n\n[법률조항]\n{article}"

        final_docs.append(Document(page_content=content, metadata=metadata))


In [19]:
        
len(final_docs)

print(final_docs[0].page_content)

print(final_docs[0].metadata)

print(final_docs[1].page_content)

print(final_docs[1].metadata)



[법률정보]
다음 조항은 전자상거래소비자보호법 제1장 총칙 <개정 2012. 2. 17.>에서 발췌한 내용입니다.

[법률조항]
제1조(목적) 이 법은 전자상거래 및 통신판매 등에 의한 재화 또는 용역의 공정한 거래에 관한 사항을 규정함으로써 소
비자의 권익을 보호하고 시장의 신뢰도를 높여 국민경제의 건전한 발전에 이바지함을 목적으로 한다.
[전문개정 2012. 2. 17.]
{'source': 'data/전자상거래 등에서의 소비자보호에 관한 법률(법률)(제20302호)(20250214).pdf', 'chapter': '제1장 총칙 <개정 2012. 2. 17.>', 'name': '전자상거래소비자보호법'}
[법률정보]
다음 조항은 전자상거래소비자보호법 제1장 총칙 <개정 2012. 2. 17.>에서 발췌한 내용입니다.

[법률조항]
제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2012. 6. 1.>
1. “전자상거래”란 전자거래(「전자문서 및 전자거래 기본법」 제2조제5호에 따른 전자거래를 말한다. 이하 같다)의
방법으로 상행위(商行爲)를 하는 것을 말한다. 
2. “통신판매”란 우편ㆍ전기통신, 그 밖에 총리령으로 정하는 방법으로 재화 또는 용역(일정한 시설을 이용하거나
용역을 제공받을 수 있는 권리를 포함한다. 이하 같다)의 판매에 관한 정보를 제공하고 소비자의 청약을 받아 재 
화 또는 용역(이하 “재화등”이라 한다)을 판매하는 것을 말한다. 다만, 「방문판매 등에 관한 법률」 제2조제3호에 
따른 전화권유판매는 통신판매의 범위에서 제외한다. 
3. “통신판매업자”란 통신판매를 업(業)으로 하는 자 또는 그와의 약정에 따라 통신판매업무를 수행하는 자를 말한
다. 
4. “통신판매중개”란 사이버몰(컴퓨터 등과 정보통신설비를 이용하여 재화등을 거래할 수 있도록 설정된 가상의 영
업장을 말한다. 이하 같다)의 이용을 허락하거나 그 밖에 총리령으로 정하는 방법으로 거래 당사자 간의 통신판매 
를 알선하는 행위를 말한다. 
5. “소비자”란

In [20]:

# 벡터저장소에 인덱싱

# 각 문서의 텍스트 길이를 확인

text_lengths = [len(d.page_content) for d in final_docs]
print(min(text_lengths), max(text_lengths))



164 2289


In [21]:

from langchain_chroma import Chroma
from langchain_ollama  import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3") 

# Chroma 인덱스 생성
labor_db = Chroma.from_documents(
    documents=final_docs, 
    embedding=embeddings_model,   
    collection_name="ecommerce_law",
    persist_directory="../chroma_db",
)




### 도로교통법

In [22]:
from langchain_community.document_loaders import PyPDFLoader

pdf_file = 'data/도로교통법(법률)(제20375호)(20250320).pdf'

loader = PyPDFLoader(pdf_file)
pages = loader.load()

len(pages)


72

In [23]:


pages[0].page_content

# 각 페이지의 텍스트를 결합하여 재분리
text_for_delete = r"법제처\s+\d+\s+국가법령정보센터\n도로교통법"

law_text = "\n".join([re.sub(text_for_delete, "", p.page_content).strip() for p in pages])

parsed_law = parse_law(law_text)

# 분할된 아이템 갯수 확인
print(len(parsed_law["장"]))

parsed_law


14


{'서문': '도로교통법 \n[시행 2025. 3. 20.] [법률 제20375호, 2024. 3. 19., 일부개정] \n경찰청 (교통기획계(법제총괄, 안전표지, 보호구역)) 02-3150-2251 \n경찰청 (교통안전계(안전, 단속, 어린이통학버스)) 02-3150-2252 \n경찰청 (운전면허계(운전면허)) 02-3150-2253 \n경찰청 (첨단교통계(신호, 무인단속장비)) 02-3150-2851 \n경찰청 (교통조사계(교통사고조사)) 02-3150-2552',
 '장': {'제1장 총칙': ['제1조(목적) 이 법은 도로에서 일어나는 교통상의 모든 위험과 장해를 방지하고 제거하여 안전하고 원활한 교통을 확보\n함을 목적으로 한다.',
   '제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2012. 3. 21., 2013. 3. 23., 2014. 1. 28., 2014. 11. 19.,\n2017. 3. 21., 2017. 7. 26., 2017. 10. 24., 2018. 3. 27., 2020. 5. 26., 2020. 6. 9., 2020. 12. 22., 2021. 10. 19., 2022. 1.\n11., 2023. 4. 18., 2023. 10. 24.>\n1. “도로”란 다음 각 목에 해당하는 곳을 말한다.\n가. 「도로법」에 따른 도로 \n나. 「유료도로법」에 따른 유료도로 \n다. 「농어촌도로 정비법」에 따른 농어촌도로 \n라. 그 밖에 현실적으로 불특정 다수의 사람 또는 차마(車馬)가 통행할 수 있도록 공개된 장소로서 안전하고 원활 \n한 교통을 확보할 필요가 있는 장소 \n2. “자동차전용도로”란 자동차만 다닐 수 있도록 설치된 도로를 말한다.\n3. “고속도로”란 자동차의 고속 운행에만 사용하기 위하여 지정된 도로를 말한다.\n4. “차도”(車道)란 연석선(차도와 보도를 구분하는 돌 등으로 이어진 선을 말한다. 이하 같다), 안전표지 또는 그와 비\n슷한 인공구조물을 이용하여 경계(境界)를 표시하여 모든 차가

In [24]:


# 랭체인 Document 객체에 메타데이터와 함께 정리
from langchain_core.documents import Document

final_docs = []
for law in parsed_law['장'].keys():
    for article in parsed_law['장'][law]:

        # metadata 내용을 정리 
        metadata = {
                "source": pdf_file,
                "chapter": law,
                "name" : "도로교통법"
                }

        # metadata 내용을 본문에 추가 
        content = f"[법률정보]\n다음 조항은 {metadata['name']} {metadata['chapter']}에서 발췌한 내용입니다.\n\n[법률조항]\n{article}"

        final_docs.append(Document(page_content=content, metadata=metadata))
 

In [25]:
       
len(final_docs)

print(final_docs[0].page_content)
print()
print(final_docs[0].metadata)

print(final_docs[1].page_content)
print()
print(final_docs[1].metadata)



[법률정보]
다음 조항은 도로교통법 제1장 총칙에서 발췌한 내용입니다.

[법률조항]
제1조(목적) 이 법은 도로에서 일어나는 교통상의 모든 위험과 장해를 방지하고 제거하여 안전하고 원활한 교통을 확보
함을 목적으로 한다.

{'source': 'data/도로교통법(법률)(제20375호)(20250320).pdf', 'chapter': '제1장 총칙', 'name': '도로교통법'}
[법률정보]
다음 조항은 도로교통법 제1장 총칙에서 발췌한 내용입니다.

[법률조항]
제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2012. 3. 21., 2013. 3. 23., 2014. 1. 28., 2014. 11. 19.,
2017. 3. 21., 2017. 7. 26., 2017. 10. 24., 2018. 3. 27., 2020. 5. 26., 2020. 6. 9., 2020. 12. 22., 2021. 10. 19., 2022. 1.
11., 2023. 4. 18., 2023. 10. 24.>
1. “도로”란 다음 각 목에 해당하는 곳을 말한다.
가. 「도로법」에 따른 도로 
나. 「유료도로법」에 따른 유료도로 
다. 「농어촌도로 정비법」에 따른 농어촌도로 
라. 그 밖에 현실적으로 불특정 다수의 사람 또는 차마(車馬)가 통행할 수 있도록 공개된 장소로서 안전하고 원활 
한 교통을 확보할 필요가 있는 장소 
2. “자동차전용도로”란 자동차만 다닐 수 있도록 설치된 도로를 말한다.
3. “고속도로”란 자동차의 고속 운행에만 사용하기 위하여 지정된 도로를 말한다.
4. “차도”(車道)란 연석선(차도와 보도를 구분하는 돌 등으로 이어진 선을 말한다. 이하 같다), 안전표지 또는 그와 비
슷한 인공구조물을 이용하여 경계(境界)를 표시하여 모든 차가 통행할 수 있도록 설치된 도로의 부분을 말한다. 
5. “중앙선”이란 차마의 통행 방향을 명확하게 구분하기 위하여 도로에 황색 실선(實線)이나 황색 점선 등의 안전표
지로 표시한 선 또는 중앙분리대나 울타리

In [26]:

# 벡터저장소에 인덱싱

# 각 문서의 텍스트 길이를 확인

text_lengths = [len(d.page_content) for d in final_docs]
print(min(text_lengths), max(text_lengths))



62 5702


In [27]:

from langchain_chroma import Chroma
from langchain_ollama  import OllamaEmbeddings

embeddings_model = OllamaEmbeddings(model="bge-m3") 

# Chroma 인덱스 생성
labor_db = Chroma.from_documents(
    documents=final_docs, 
    embedding=embeddings_model,   
    collection_name="road_traffic_law",
    persist_directory="../chroma_db",
)